<a href="https://colab.research.google.com/github/cnewton1428/DataAnalytics/blob/main/Module_9_Decision_Trees_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [32]:
from google.colab import files
uploaded = files.upload()

Saving titanic.xls to titanic.xls


In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [34]:
import io
df = pd.read_excel(io.BytesIO(uploaded['titanic.xls']))

In [35]:

df.head()


pclass  survived  ...   body                        home.dest
0       1         1  ...    NaN                     St Louis, MO
1       1         1  ...    NaN  Montreal, PQ / Chesterville, ON
2       1         0  ...    NaN  Montreal, PQ / Chesterville, ON
3       1         0  ...  135.0  Montreal, PQ / Chesterville, ON
4       1         0  ...    NaN  Montreal, PQ / Chesterville, ON

[5 rows x 14 columns]

### Clean the data

In [36]:
#find columns that have missing values
df.isnull().sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

In [41]:
#fill missing values for age based on survival status, sex, and passenger class
df['age'].fillna(df.groupby(['survived', 'sex', 'pclass'])['age'].transform('mean'), inplace=True)

In [40]:
#only 2 missing values so we'll fill with most common embarkation point
df['embarked'].value_counts()

S    914
C    270
Q    123
Name: embarked, dtype: int64

In [42]:
#fill missing values
df['embarked'].fillna('S', inplace=True)

In [43]:
df.isnull().sum()

pclass          0
survived        0
name            0
sex             0
age             0
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        0
boat          823
body         1188
home.dest     564
dtype: int64

In [54]:
modeldf = df.drop(['name','ticket','fare', 'cabin', 'boat', 'body', 'home.dest'], axis=1)

In [55]:
#columns left in our dataframe
modeldf.columns

Index(['pclass', 'survived', 'sex', 'age', 'sibsp', 'parch', 'embarked'], dtype='object')

Create dummy variables for categorical values

In [56]:
#dummy variables for passenger class embarkation port
#get_dummies will auto-drop columns that dummies were created from
modeldf = pd.get_dummies(data=modeldf, columns=['pclass','embarked'])
modeldf.head()

survived     sex      age  ...  embarked_C  embarked_Q  embarked_S
0         1  female  29.0000  ...           0           0           1
1         1    male   0.9167  ...           0           0           1
2         0  female   2.0000  ...           0           0           1
3         0    male  30.0000  ...           0           0           1
4         0  female  25.0000  ...           0           0           1

[5 rows x 11 columns]

In [57]:
#change sex values to binary
#female=0, male=1
modeldf['sex'] = modeldf['sex'].map({'female':0, 'male':1})
modeldf.head()

survived  sex      age  sibsp  ...  pclass_3  embarked_C  embarked_Q  embarked_S
0         1    0  29.0000      0  ...         0           0           0           1
1         1    1   0.9167      1  ...         0           0           0           1
2         0    0   2.0000      1  ...         0           0           0           1
3         0    1  30.0000      1  ...         0           0           0           1
4         0    0  25.0000      1  ...         0           0           0           1

[5 rows x 11 columns]

In [58]:
#create new column based on number of family members
#drop sibsp and parch columns
modeldf['family_num'] = modeldf['sibsp'] + modeldf['parch']
modeldf.drop(['sibsp', 'parch'], axis=1, inplace=True)
modeldf.head()

survived  sex      age  ...  embarked_Q  embarked_S  family_num
0         1    0  29.0000  ...           0           1           0
1         1    1   0.9167  ...           0           1           3
2         0    0   2.0000  ...           0           1           3
3         0    1  30.0000  ...           0           1           3
4         0    0  25.0000  ...           0           1           3

[5 rows x 10 columns]

In [59]:
modeldf['TravelAlone']=np.where((modeldf['family_num'] > 0), 0, 1)
modeldf.head()

survived  sex      age  ...  embarked_S  family_num  TravelAlone
0         1    0  29.0000  ...           1           0            1
1         1    1   0.9167  ...           1           3            0
2         0    0   2.0000  ...           1           3            0
3         0    1  30.0000  ...           1           3            0
4         0    0  25.0000  ...           1           3            0

[5 rows x 11 columns]

### Build a Decision Tree

In [61]:
#extract target variable
#make copy of 'survived' column
y = modeldf['survived']

In [62]:
#copy of modeldf without 'survived' column
X = modeldf.drop(['survived'], axis=1)

In [64]:
#80% for training data, 20% for test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=90)

In [66]:
#assign decision tree function to model variable
model = tree.DecisionTreeClassifier()

In [67]:
#develop model using training data
#defining arguments in the model can help prevent overfitting
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [68]:
#run the predictions on the test data
y_predict = model.predict(X_test)

In [69]:
#check the accuracy of model
accuracy_score(y_test, y_predict)

0.8091603053435115

In [70]:
#look at true and false predictions
pd.DataFrame(
    confusion_matrix(y_test, y_predict),
    columns=['Predicted Not Survival', 'Predicted Survival'],
    index=['True Not Survival', 'True Survival']
)

Predicted Not Survival  Predicted Survival
True Not Survival                     145                  22
True Survival                          28                  67